# Stock Market Portfolio Generator

![](banner_project.jpg)

In [2]:
f = "setup.R"; for (i in 1:10) { if (file.exists(f)) break else f = paste0("../", f) }; source(f)
options(repr.matrix.max.rows=674)
options(repr.matrix.max.cols=200)
update_geom_defaults("point", list(size=1))                                

.................................................... start of document ....................................................

## Overview

### Objective

Recommend a portfolio of 12 company investments that maximizes 12-month return of an overall $1,000,000 investment made at the end of quarter 4 of year 2018.

### Approach

* Explore company fundamentals data for thousands of companies from quarters 1, 2, 3, and 4 of year 2017 + company stock price data for those companies from quarter 4 of year 2018.


* Explore various ways of representing the data, using variable selection, principal component analysis, cluster analysis, and other methods.


* Construct models to predict 12-month stock price growth given 12 months of past company fundamentals data, using various machine learning model construction methods.


* Develop criteria to choose company investment levels informed by the models, and estimate the models' business performance.


* Tune the models for optimal performance and select the best one.


* Use the best model applied to company fundamentals data for a set of investment opportunities for quarters 1, 2, 3, and 4 of year 2018, and accordingly recommend a portfolio of 12 company investments.

### Data

**Datasets:**<br>
Datasets comprise a list of company investment opportunities, a data dictionary, and company fundamentals for select US active, publicly held companies that reported quarterly measures including stock prices for 1st, 2nd, 3rd, and 4th quarters in years 2017 and 2018.  All non-missing stock prices exceed $3 per share.  File formats are all comma-separated values (CSV).

* Data Dictionary.csv
* Company Fundamentals 2017.csv
* Company Fundamentals 2018.csv
* Investment Opportunities.csv


**Data Source:**<br>
Wharton Research Data Services > Compustat - Capital IQ from Standard & Poor's > North America - Daily > Fundamentals Quarterly (https://wrds-www.wharton.upenn.edu/)

* Date Variable: Data Date
* Date Range: 2017-01 to 2017-12 -or- 2018-01 to 2018-12
* Company Codes: Search the entire database
  * Consolidtaion Level: C, Output
  * Industry Format: INDL, FS, Output
  * Data Format: STD, Output
  * Population Source: D, Output
  * Quarter Type: Fiscal View, Output
  * Currency: USD, Output (not CAD)
  * Company Status: Active, Output (not Inactive)
* Variable Types: Data Items, Select All (674)
* Query output:
  * Output format: comma-delimited text
  * Compression type: None
  * Data format: MMDDYY10
  
  
* For Data Dictionary: Variable Descriptions tab, copy to excel, save as csv

<br>



## Business Model

Business model is ...

$$ \begin{align} profit = \left( \sum_{i \in portfolio} (1 + growth_i) \times allocation_i \right) - budget \end{align} $$

<br>

$$ profit\,rate = profit \div budget $$


$$ \begin{align} budget = \sum_{i \in portfolio} allocation_i \end{align} $$


Business model parameters include ...

* Budget = \$1,000,000: total investment to allocate across the companies in the portfolio
* Portfolio Size = 12: number of companies in the portfolio



In [3]:
# Setting business parameters.
budget = 1000000
portfolio_size = 12
threshold = .3

## Data Retrieval



### Data Dictionary

Let's read the data and see how many entries we are working with

In [5]:
# Retrieving the data dictionary.
dict = read.csv('Data Dictionary.csv')
fund_17 = read.csv('Company Fundamentals 2017.csv')
fund_18 = read.csv('Company Fundamentals 2018.csv')
uniques = c(unique(dict$Variable.Name), unique(dict$Data.Type), unique(dict$Variable.Description), unique(dict$Help))
classes = c(class(dict$Variable.Name), class(dict$Data.Type), class(dict$Variable.Description), class(dict$Help))

sprintf("There are %s entries in Company Fundamentals 2017 and %s entries in Company Fundamentals 2018", nrow(fund_17), nrow(fund_18))
layout(fmt(size(fund_17)), fmt(size(fund_18)))

[1] "There are 33269 entries in Company Fundamentals 2017 and 35728 entries in Company Fundamentals 2018"

observations,variables
33269,680
observations,variables
35728,680


### Data for 2017

#### Partition Data by Calendar Quarter 

To partition the dataset by calendar quarter in which information is reported, I'll first add a synthetic variable to indicate such.  I'll then partition into four new datasets, one for each quarter, and drop the quarter variables. Additionally, I'll filter the observations to include only those with non-missing `prccq` $\geq$ 3.  Then, I'll remove any observations about companies that reported more than once per quarter.  Finally, I'll change all the variable names (except for the `gvkey`, `tic`, and `conm` variables) by suffixing them with quarter information - e.g., in the Quarter 1 dataset, `prccq` becomes `prccq.q1`, etc.

In [6]:
# Partitioning the dataset as described.
datax = fund_17
datax$quarter = quarter(mdy(datax[,2]))
 
data.current.q1 = datax[(datax$quarter==1) & !is.na(datax$prccq) & (datax$prccq>=3), -ncol(datax)]
data.current.q2 = datax[(datax$quarter==2) & !is.na(datax$prccq) & (datax$prccq>=3), -ncol(datax)]
data.current.q3 = datax[(datax$quarter==3) & !is.na(datax$prccq) & (datax$prccq>=3), -ncol(datax)]
data.current.q4 = datax[(datax$quarter==4) & !is.na(datax$prccq) & (datax$prccq>=3), -ncol(datax)]
 
data.current.q1 = data.current.q1[!duplicated(data.current.q1$gvkey),]
data.current.q2 = data.current.q2[!duplicated(data.current.q2$gvkey),]
data.current.q3 = data.current.q3[!duplicated(data.current.q3$gvkey),]
data.current.q4 = data.current.q4[!duplicated(data.current.q4$gvkey),]
 
colnames(data.current.q1)[-c(1, 10, 12)] = paste0(colnames(data.current.q1)[-c(1, 10, 12)], ".q1")
colnames(data.current.q2)[-c(1, 10, 12)] = paste0(colnames(data.current.q2)[-c(1, 10, 12)], ".q2")
colnames(data.current.q3)[-c(1, 10, 12)] = paste0(colnames(data.current.q3)[-c(1, 10, 12)], ".q3")
colnames(data.current.q4)[-c(1, 10, 12)] = paste0(colnames(data.current.q4)[-c(1, 10, 12)], ".q4")


#### Consolidate Data by Company

Consolidate the four quarter datasets into one dataset, with one observation per company that includes variables for all four quarters.  Remove any observations with missing `prccq.q4` values.

In [8]:
# Consolidating the partitions as described.
m12 = merge(data.current.q1, data.current.q2, by=c("gvkey", "tic", "conm"), all=TRUE)
m34 = merge(data.current.q3, data.current.q4, by=c("gvkey", "tic", "conm"), all=TRUE)
 
data.current = merge(m12, m34, by=c("gvkey", "tic", "conm"), all=TRUE, sort=TRUE)
data.current = data.current[!is.na(data.current$prccq.q4),]
 
fmt(size(data.current))

observations,variables
4434,2711


### Data for 2018

#### Filter Data by Calendar Quarter 4 

To filter the dataset by calendar quarter in which information is reported, I'll first add a synthetic variable to indicate such, and then select only observations with information reported in quarter 4. Additionally, I'll filter the observations to include only those with non-missing `prccq`, and keep only the `gvkey` and `prccq` variables.  Then, I'll remove any observations about companies that reported more than once per quarter.

In [9]:
# Filtering the dataset as described.
datax = fund_18
datax$quarter = quarter(mdy(datax[,2]))

data.future.q4 = datax[(datax$quarter==4) & !is.na(datax$prccq), c("gvkey", "prccq")]
data.future.q4 = data.future.q4[!duplicated(data.future.q4$gvkey),]
data.future.q4[1:6,]
size(data.future.q4)

,gvkey,prccq
4,1004,43.69
8,1045,32.11
12,1050,6.75
16,1062,8.66
20,1072,15.25
24,1075,85.20


observations,variables
5968,2


### Ready Data for Analysis

Here, I'll consolidate the processed 2017 dataset and processed 2018 dataset, keeping only observations that have both 2017 and 2018 information.  Then, I'll add these 2 synthetic variables:

$\begin{align}
growth : & \, (prccq - prccq.q4) \div prccq.q4 \\
big\_growth : & \, growth \geq threshold
\end{align}$

<br>

$prccq$, $growth$, and $big\_growth$ are outcome variables.  All others are predictor variables.

In [11]:
# Consolidating the datasets as described.
fund = merge(data.current, data.future.q4)
fund$growth = (fund$prccq - fund$prccq.q4) / fund$prccq.q4
fund$big_growth = fund$growth >= threshold
sprintf("There are %s different variables. Which is 3 outcome variables and 2711 predictor variables", length(colnames(fund)))
fmt(size(fund))

[1] "There are 2714 different variables. Which is 3 outcome variables and 2711 predictor variables"

observations,variables
4305,2714


## New Representation of Data


### Variable Selection

Here, I'll filter the data to include only predictor variables with at least 80% non-missing values.

In [12]:
# Selecting variables.
data.filtered = fund[ ,!(colnames(fund) == "growth")]
data.filtered = data.filtered[ ,!(colnames(data.filtered) == "prccq.q1")]
data.filtered = data.filtered[ ,!(colnames(data.filtered) == "big_growth")]
data.filtered = data.filtered[, -which(colMeans(is.na(data.filtered)) > 0.2)]
size(data.filtered)

observations,variables
4305,923


### Imputation

In this cell, I'll impute missing data: for each numeric variable, I'll use the mean of non-missing values; for each non-numeric variable, I'll use the mode of non-missing values.

In [13]:
# Imputing missing data.
imputed = get_impute(data.filtered)
data.x = data.filtered
data.filtered = impute(data.filtered)
size(data.filtered)

observations,variables
4305,923


### Principal Component Analysis

In this section, I'll perform PCA. In preparation for principal component analysis, I'll filter the data to include only numeric and integer variables with non-zero variance.

In [14]:
# Filtering data for PCA
data.filtered.num = Filter(is.numeric, data.filtered)
data.filt = data.filtered.num[ - as.numeric(which(apply(data.filtered.num, 2, var) == 0))]
size(data.filt)

observations,variables
4305,737


In [18]:
# Performing principal component analysis.
pc = prcomp(data.filt, scale=TRUE)
princ = pc$x
head(princ)


PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35,PC36,PC37,PC38,PC39,PC40,PC41,PC42,PC43,PC44,PC45,PC46,PC47,PC48,PC49,PC50,PC51,PC52,PC53,PC54,PC55,PC56,PC57,PC58,PC59,PC60,PC61,PC62,PC63,PC64,PC65,PC66,PC67,PC68,PC69,PC70,PC71,PC72,PC73,PC74,PC75,PC76,PC77,PC78,PC79,PC80,PC81,PC82,PC83,PC84,PC85,PC86,PC87,PC88,PC89,PC90,PC91,PC92,PC93,PC94,PC95,PC96,PC97,PC98,PC99,PC100,⋯,PC638,PC639,PC640,PC641,PC642,PC643,PC644,PC645,PC646,PC647,PC648,PC649,PC650,PC651,PC652,PC653,PC654,PC655,PC656,PC657,PC658,PC659,PC660,PC661,PC662,PC663,PC664,PC665,PC666,PC667,PC668,PC669,PC670,PC671,PC672,PC673,PC674,PC675,PC676,PC677,PC678,PC679,PC680,PC681,PC682,PC683,PC684,PC685,PC686,PC687,PC688,PC689,PC690,PC691,PC692,PC693,PC694,PC695,PC696,PC697,PC698,PC699,PC700,PC701,PC702,PC703,PC704,PC705,PC706,PC707,PC708,PC709,PC710,PC711,PC712,PC713,PC714,PC715,PC716,PC717,PC718,PC719,PC720,PC721,PC722,PC723,PC724,PC725,PC726,PC727,PC728,PC729,PC730,PC731,PC732,PC733,PC734,PC735,PC736,PC737
3.437026,-0.2214612,0.42173751,-0.6326742,1.5547346,-0.5654384,2.0475876,-4.7934517,2.57730773,1.08458798,-1.0862641,-0.7297127,0.2969840,6.8506191,-1.40348828,1.0990780,0.1814650,0.9899768,-0.79304191,0.19916007,-0.3167686,1.0530731,1.67811387,-0.8667728,-0.37924724,-1.16145300,1.0876456,-1.0402680,0.8550118,-0.1308677,-0.9077599,0.2754206,-0.72754324,-0.04243444,-1.0435284,0.37192500,0.09528258,1.00867366,-0.6293308,0.55889783,-0.13176967,0.52259389,-1.7070397,2.7192290,-0.34168959,0.22828921,-0.09991265,-0.57794811,-0.2900903,0.2200521,-0.47605676,-0.1951741,-0.05089903,-0.07024937,0.4389557,-0.2722637,-0.0987418064,-0.31595011204,-0.6649850,0.59204523,0.48727964,0.3470977,0.29489569,-0.283226674,-0.4221029033,-0.09672700,-0.074176035,0.57985545,0.22562304,-0.96556143,0.26177436,1.48999300,-0.24223471,-0.840735798,1.0517995,-1.50630962,-2.633578488,-1.49749046,-4.00246257,-0.82597660,-3.83926181,-1.85721178,1.62861480,-0.58109160,-2.41890422,0.82976699,0.002919377,1.9151847,-1.63955262,-0.61220391,0.3054356,1.62475884,-0.2848171,0.19355465,0.15458417,0.44065724,0.1104779,-0.9759045,-0.1275174,-0.46725093,⋯,0.00026890976,-0.00023635091,0.00044341450,0.000090364199,0.00023843817,-0.00005206102,0.00003903972,-0.00007387230,-0.000157449675,-0.000138641626,-0.00002854977,0.000042287001,0.000292705013,0.00032473936,0.00014567185,0.000146448396,-0.000170107920,-0.00003448985,0.00028558601,0.00016365036,-0.000154234510,-0.00029234046,-0.000100922050,-0.00004458205,0.00032625554,0.00008389825,-0.000011824119,-0.000025850869,0.00020615730,-0.000042683233,-0.00008624288,0.000004802560,-0.000190323006,-0.000037667508,-0.00006097613,-0.00006945291,0.000179219572,-0.000033429299,0.000029288835,-0.000012324662,0.000009052659,-0.0000123097086,-0.00001491322074,-0.0000082646511,0.0000148841842,-0.0000113819402,-0.0000023825739,0.0000158681347,-0.0000091692953,0.0000053559286,-0.000008791974,0.0000073379265,-0.0000020720397,-0.0000040715156,-0.0000020103303,-0.00000096224209,-0.0000030342634,-0.000000119544701,-0.00000065985314,-0.000000025465513,-0.00000001409388,-0.000000001776530,-0.000000018341912,0.00000000296975142,-0.000000000037619422,3.690191e-15,3.265671e-15,2.619432e-16,1.590199e-15,5.377643e-17,-3.053113e-16,-1.207801e-15,3.963843e-16,6.071532e-16,2.087631e-16,-1.208777e-15,1.444157e-16,-4.951009e-16,2.328866e-16,1.153591e-16,2.072995e-16,-8.101159e-16,-1.053845e-15,4.189357e-16,3.964927e-16,1.019150e-17,-6.245005e-17,-1.231654e-16,-2.116363e-16,-7.973223e-16,-8.187895e-16,-2.706169e-16,1.040834e-15,1.106754e-15,7.754214e-16,-2.735225e-15,3.851086e-16,5.802650e-16,2.246467e-16,4.163336e-17
-12.033145,0.8010104,5.91825772,-0.8021252,6.7209209,-3.0465557,0.3474168,-0.9159117,-0.08654886,-3.11236549,-2.0995968,-0.6278466,2.3324190,-1.9084853,0.31186074,0.6937267,-0.7265996,0.6662153,-0.96433985,0.17794874,-1.2926844,1.3592125,0.94050043,1.0356987,0.75765557,-0

### Cluster Analysis

In the cell below, I'll perform cluster analysis, and assign each observation a cluster, labeled 1-4.

In [26]:
# Performing a cluster analysis.
data.d = cbind(dummify(data.filtered[,'incorp.q1', drop=FALSE]), dummify(data.filtered[,'state.q1', drop=FALSE]))
data.d$incorp.q1 = NULL
data.d$state.q1 = NULL
set.seed(12345)
data.d$cluster = kmeans(data.d, centers=4)$cluster


### Transform the Data

Finally, I'll add cluster, PC1, PC2, and growth to the data.

In [27]:
# Adding in cluster, PC1, PC2, and growth.

data.filtered$cluster = data.d$cluster
data.filtered$PC1 = princ[,'PC1']
data.filtered$PC2 = princ[,'PC2']


data.filtered$growth = fund$growth
size(data.filtered)

observations,variables
4305,927


## Model for Prediction

### Build Model

Here we train a linear model to predict growth given PC1, PC2, and cluster.

In [28]:
# Constructing a model to predict growth or big_growth.

model = lm(growth ~ PC1+PC2+cluster, data.filtered)
data.frame(model$coefficients)

,model.coefficients
(Intercept),-0.1462353728
PC1,0.0001644893
PC2,0.0006777451
cluster,0.0124914774


### In-Sample Estimated Performance

Let's take a look at our estimated in-sample performance, or profit.

In [40]:
# Presenting the model's in-sample estimated profit and profit rate.
cutoff = .6
prob = predict(model, data.filtered)

data.filtered$prediction = prob
data.ordered = data.filtered[order(-data.filtered$prediction),]
profit = mean(data.ordered[1:12, 'prediction']) * budget
profit

[1] -46714.42

Interesting, our model predicts a negative number. This might be due to a problem with the intercept. 

## Model Summary



In [59]:
summary(model)


Call:
lm(formula = growth ~ PC1 + PC2 + cluster, data = data.filtered)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.8872 -0.2176 -0.0262  0.1413 10.3520 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.1462354  0.0157594  -9.279   <2e-16 ***
PC1          0.0001645  0.0005589   0.294   0.7685    
PC2          0.0006777  0.0009705   0.698   0.4850    
cluster      0.0124915  0.0063466   1.968   0.0491 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4688 on 4301 degrees of freedom
Multiple R-squared:  0.001057,	Adjusted R-squared:  0.0003606 
F-statistic: 1.518 on 3 and 4301 DF,  p-value: 0.2078


## Deploying the Model

### Retrieve Investment Opportunities

In [60]:
data.opportunities = read.csv("Investment Opportunities.csv", header=TRUE)
size(data.opportunities)
data.opportunities[1:6,]

observations,variables
230,3


gvkey,tic,conm
1004,AIR,AAR CORP
10549,THO,THOR INDUSTRIES INC
106133,DSU,BLACKROCK DEBT STRATEGIES FD
10618,TTC,TORO CO
11178,VLGEA,VILLAGE SUPER MARKET -CL A
111873,RH,RH


### Selected Data for 2018

#### Retrieving Raw Test Data

In [62]:
# Retrieving the 2018 data 

fund_18.f = fund_18[fund_18$gvkey %in% data.opportunities$gvkey,]
data.filtered.f = data.filtered[data.filtered$gvkey %in% data.opportunities$gvkey,]
layout(size(fund_18), size(fund_18.f), size(data.filtered), size(data.filtered.f))

observations,variables
35728,680
observations,variables
918,680
observations,variables
4305,928
observations,variables
196,928


### Applying the Model

In this cell, I'll generate a recommended a portfolio of allocations to 12 companies stored as a data.frame

In [77]:
# Presenting the portfolio recommendation.
data.ordered = data.filtered.f[order(-data.filtered.f$prediction),]
portfolio = data.ordered[1:12, c('gvkey', 'tic', 'conm')]
portfolio$allocation = budget / 12
head(portfolio)

,gvkey,tic,conm,allocation
174,3504,COO,COOPER COS INC (THE),83333.33
3259,147708,CVGW,CALAVO GROWERS INC,83333.33
2017,27928,INTU,INTUIT INC,83333.33
739,11511,WSM,WILLIAMS-SONOMA INC,83333.33
3018,118267,KFY,KORN FERRY,83333.33
3662,176362,AVAV,AEROVIRONMENT INC,83333.33


### Write Portfolio Recommendation

This cell will export the recommended portfolio to a csv file.

In [78]:
write.csv(portfolio, paste0(analyst, ".csv"), row.names=FALSE)